In [1]:
import sys, os

env_root = '/N/project/baby_vision_curriculum/pythonenvs/hfenv/lib/python3.10/site-packages/'
sys.path.insert(0, env_root)

In [2]:
import numpy as np
# import torch, torchvision
# from torchvision import transforms as tr
from tqdm import tqdm
from pathlib import Path
# import math
import argparse
import pandas as pd
import warnings

from copy import deepcopy

from sklearn import svm, preprocessing
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

from joblib import Parallel, delayed

In [3]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
import json 
import logging
logger = logging.getLogger(__name__)

In [4]:
logger

<Logger __main__ (WARNING)>

In [5]:
def topk_retrieval(feature_dir, cfg):
    """Extract features from test split and search on train split features."""
    logger.info('Loading local .npy files...')
    fold = cfg.get_int('dataset.fold')

    X_train = np.load(os.path.join(feature_dir, f'train_fold{fold}_feats.npy'))
    y_train = np.load(os.path.join(feature_dir, f'train_fold{fold}_labels.npy'))

    X_test = np.load(os.path.join(feature_dir, f'test_fold{fold}_feats.npy'))
    y_test = np.load(os.path.join(feature_dir, f'test_fold{fold}_labels.npy'))

    ks = [1, 5, 10, 20, 50]
    topk_correct = {k:0 for k in ks}

    distances = cosine_distances(X_test, X_train)
    indices = np.argsort(distances)

    for k in ks:
        # print(k)
        top_k_indices = indices[:, :k]
        # print(top_k_indices.shape, y_test.shape)
        for ind, test_label in zip(top_k_indices, y_test):
            labels = y_train[ind]
            if test_label in labels:
                # print(test_label, labels)
                topk_correct[k] += 1

    for k in ks:
        correct = topk_correct[k]
        total = len(X_test)
        logger.info('Top-{}, correct = {:.2f}, total = {}, acc = {:.3f}'.format(k, correct, total, correct/total))

    with open(os.path.join(feature_dir, f'topk_correct_fold{fold}.json'), 'w') as fp:
        json.dump(topk_correct, fp)
        
def get_separability_score_old(df, label, method='sgd', ret_preds=False):
    # method: sgd or svm
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(df[label])
    
    X_cols = ['dim'+str(i)
              for i in range(768)]
    X = df[X_cols]
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    if method=='svm':
        clf = make_pipeline(StandardScaler(),
                            LinearSVC(random_state=0, tol=1e-4))
    elif method=='sgd':
        clf = make_pipeline(StandardScaler(),
                            SGDClassifier(max_iter=5000, tol=1e-4, n_jobs=20))#, loss='log_loss'))
    else:
        raise ValueError()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train,y_train)
    test_score = clf.score(X_test,y_test)
    if ret_preds:
        preds = clf.predict(X_test)
        return train_score, test_score, preds, y_test
    return train_score, test_score

In [6]:
def get_nn_score(df_train, df_test, label, metric='cosine',
                 savedir=None, run_id=None):
#     Get nearest neighbor score
    
    le = preprocessing.LabelEncoder()
    y_train = le.fit_transform(df_train[label])
    
    X_cols = [col for col in df_train.columns if 'dim' in col]
    x_train = df_train[X_cols]
    
    if df_test is not None:
        x_test = df_test[X_cols]
        y_test = le.transform(df_test[label])
    
    ks = [1, 5, 10, 20, 50]
    topk_correct = {k:0 for k in ks}

    if metric=='cosine':
        distances = cosine_distances(x_test, x_train)
    else:
        distances = euclidean_distances(x_test, x_train)
    indices = np.argsort(distances)
    
    for k in ks:
        # print(k)
        top_k_indices = indices[:, :k]
        # print(top_k_indices.shape, y_test.shape)
        for ind, test_label in zip(top_k_indices, y_test):
            labels = y_train[ind]
            if test_label in labels:
                # print(test_label, labels)
                topk_correct[k] += 1
        topk_correct[k] = topk_correct[k]/len(y_test)
    
    for k in ks:
        correct = topk_correct[k]
        total = len(x_test)
        logger.info('Top-{}, correct = {:.2f}, total = {}, acc = {:.3f}'.format(k, correct, total, correct/total))
        print('Top-{}, correct = {:.2f}, total = {}, acc = {:.3f}'.format(k, correct, total, correct/total))

    if savedir is not None:
        if run_id is None:
            raise ValueError
            # run_id = ''
        with open(os.path.join(savedir, 
                               run_id+'_topk_correct.json'), 'w') as fp:
            json.dump(topk_correct, fp)

    return topk_correct#[1], topk_correct[5], topk_correct[10]

def get_separability_score(df_train, df_test, label, 
                           method='sgd', ret_preds=False,
                          n_jobs=80):
    # method: sgd or svm
    le = preprocessing.LabelEncoder()
    y_train = le.fit_transform(df_train[label])
    
    X_cols = [col for col in df_train.columns if 'dim' in col]
    x_train = df_train[X_cols]
    
    if df_test is not None:
        x_test = df_test[X_cols]
        y_test = le.transform(df_test[label])
    else:
        x_train, x_test, y_train, y_test = train_test_split(
            x_train, y_train, test_size=0.33, random_state=42)
    
    if method=='svm':
        clf = make_pipeline(StandardScaler(),
                            LinearSVC(random_state=0, tol=1e-4))
    elif method=='sgd':
        clf = make_pipeline(StandardScaler(),
                            SGDClassifier(max_iter=5000, tol=1e-4, n_jobs=n_jobs))#, loss='log_loss'))
    else:
        raise ValueError()
    clf.fit(x_train, y_train)
    train_score = clf.score(x_train,y_train)
    test_score = clf.score(x_test,y_test)
    if ret_preds:
        preds = clf.predict(x_test)
        return train_score, test_score, preds, y_test
    return train_score, test_score

In [7]:
class SSv2Eval():
    
    def __init__(self, label_paths=None):
        if label_paths is None:
            label_paths={'train':'/N/project/baby_vision_curriculum/benchmarks/ssv2/easy_labels/train_easy10.csv',
                         'test':'/N/project/baby_vision_curriculum/benchmarks/ssv2/easy_labels/val_easy10.csv'}        
        self.labels_df = {x: pd.read_csv(label_paths[x])
                         for x in ['train','test']}
        for phase in ['train', 'test']:
            self.labels_df[phase].set_index('fname', inplace=True)

    def get_categorylabel(self, fname, phase):
        return self.labels_df[phase].loc[str(fname)+'.webm', 'label']

    def add_labels_to_df(self, df, labels, phase):
        if 'category' in labels:
            df['category'] = df['fnames'].apply(
                lambda fname: self.get_categorylabel(fname, phase))
        return df
    
    def proc_train_test(self, data_fpaths, score_type, eval_type='linear', n_jobs=80):
        if score_type!='category':
            raise ValueError
        method='sgd'
#         data_fpaths = {'train':fp_train, }
        data_df = {}
        for phase in ['train', 'test']:
            data_df[phase] = pd.read_csv(
                data_fpaths[phase])
            data_df[phase] = self.add_labels_to_df(data_df[phase], ['category'], phase)
        
        if eval_type=='linear':
            train_score, test_score, preds, targets = get_separability_score(
                data_df['train'], data_df['test'], score_type, method=method, 
                ret_preds=True, n_jobs=n_jobs)
        else:
            test_score = get_nn_score(
                data_df['train'], data_df['test'], score_type, metric='cosine',
                 savedir=None, run_id=None)
        
        return test_score

In [8]:
class UCF101Eval():
    
    def __init__(self):
        pass
    
    def add_labels_to_df(self, df):
        df['category'] = df['fnames']#.apply(lambda fname: self.get_categorylabel(fname))
        return df
    
    def proc_train_test(self, data_fpaths, score_type, eval_type='linear', n_jobs=80):
        if score_type!='category':
            raise ValueError
        method='sgd'
#         data_fpaths = {'train':fp_train, }
        data_df = {}
        for phase in ['train', 'test']:
            data_df[phase] = pd.read_csv(
                data_fpaths[phase])
            data_df[phase] = self.add_labels_to_df(data_df[phase])
        
        if eval_type=='linear':
            train_score, test_score, preds, targets = get_separability_score(
                data_df['train'], data_df['test'], score_type, method=method, 
                ret_preds=True, n_jobs=n_jobs)
        else:
            test_score = get_nn_score(
                data_df['train'], data_df['test'], score_type, metric='cosine',
                 savedir=None, run_id=None)
        
        return test_score

In [9]:
# example name: airplane_01_pivothead_absent.mp4

class ToyBoxEval():
    
    def __init__(self, exemplar=True):
        self.exemplar = exemplar
#         in the categorization test, the train and test are divided by exemplars.
    
    def get_exemplar_split(self, df, test_size=0.33, random_state=42):
        
        categories_unq = df['category'].unique()
        
        train_rows, test_rows = [],[]
        for i_c, cat in enumerate(categories_unq):
            cdata = df[df['category']==cat]
            identities_unq = cdata['identity'].unique()
            identity_train, identity_test = train_test_split(identities_unq, test_size=test_size)
#             print('identity_train:',identity_train[:10],
#                   'identity_test:',identity_test[:10]) #@@@
#                                               random_state=random_state)
            train_rows.append(
                cdata[cdata['identity'].isin(identity_train)])
            test_rows.append(
                cdata[cdata['identity'].isin(identity_test)])
        train_df = pd.concat(train_rows, axis=0, ignore_index=True)
        test_df = pd.concat(test_rows, axis=0, ignore_index=True)
        
#         print('finished get_exemplar_split') #@@@
        
        return train_df, test_df
    
    
    def get_separability_score(self, df, label, method='sgd', ret_preds=False):
#         use self.get_exemplar_split

        train_df, test_df = self.get_exemplar_split(df, test_size=0.33)#, random_state=42)

        le = preprocessing.LabelEncoder().fit(df[label])
        X_cols = sorted([col for col in df.columns if 'dim' in col])
        
        y_train = le.transform(train_df[label])
        y_test = le.transform(test_df[label])
        
        X_train, X_test = train_df[X_cols], test_df[X_cols]

#         print('train_df[label]',train_df[label].iloc[::100])
#         print('x_train.shape', X_train.shape)
        
        if method=='svm':
            clf = make_pipeline(StandardScaler(),
                                LinearSVC(random_state=0, tol=1e-4))
        elif method=='sgd':
            clf = make_pipeline(StandardScaler(),
                                SGDClassifier(max_iter=5000, tol=1e-4, n_jobs=20))#, loss='log_loss'))
        else:
            raise ValueError()
        clf.fit(X_train, y_train)
        train_score = clf.score(X_train,y_train)
        test_score = clf.score(X_test,y_test)
        if ret_preds:
            preds = clf.predict(X_test)
            return train_score, test_score, preds, y_test
        return train_score, test_score

        
    def get_labels(self, fname):
    #     example fname: truck_30_pivothead_rzminus.mp4
        fnparts = fname.split('.')[0].split('_')
        category = fnparts[0]
        identity = category+fnparts[1]
        trans = fnparts[3]
        return category, identity, trans

    def get_categorylabel(self, fname):
        fnparts = fname.split('.')[0].split('_')
        category = fnparts[0]
        return category

    def get_identitylabel(self, fname):
        fnparts = fname.split('.')[0].split('_')
        category = fnparts[0]
        identity = category+fnparts[1]
        return identity

    def get_translabel(self, fname):
        fnparts = fname.split('.')[0].split('_')
        translabel = fnparts[3]
        return translabel

    def add_labels_to_df(self, df, labels):
        if 'category' in labels:
            df['category'] = df['fnames'].apply(lambda fname: self.get_categorylabel(fname))
        if 'identity' in labels:
            df['identity'] = df['fnames'].apply(lambda fname: self.get_identitylabel(fname))
        if 'transformation' in labels:
            df['transformation'] = df['fnames'].apply(lambda fname: self.get_translabel(fname))
        return df
    
    def fix_fnames(self, df):
        df.iloc[2163,0] = 'giraffe_02_pivothead_rzplus.mp4'
        df.iloc[1851,0]='duck_06_pivothead_rxminus.mp4'
        df.iloc[751,0] = 'car_03_pivothead_rzminus.mp4'
        return df
    
    def proc_fp(self, fp, score_type):
        df = pd.read_csv(fp)
        df = self.fix_fnames(df)
        
        df = self.add_labels_to_df(df, ['category', 'identity','transformation'])
        method='sgd'
        
        if (score_type=='category') and self.exemplar:
            train_score, test_score, preds, targets = self.get_separability_score(
                df, score_type, method=method, ret_preds=True)
        else:
#             use the generic method of get_separability_score
            train_score, test_score, preds, targets = get_separability_score(
                df, None, score_type, method=method, ret_preds=True)
        return test_score

In [10]:
def get_traingroups(curr, stage):
    if curr=='dev':
        return 'g0g1g2'[:2*stage]
    elif curr=='adev':
        return 'g2g1g0'[:2*stage]
    else:
        return 'na'
    
def parse_fname(fp):
# embeddings_adev_1_g2_default_0_246.csv'
    parts = Path(fp).stem.split('_')
    if len(parts)<7:
        print('Filename does not match the format')
        if 'na' in parts:
            prefix, curr, stage, current_gr, cond, fold, seed = \
                'embeddings', 'untrained', '0', 'na', 'na', '0', '0' 
#             train_gr = 'na'
    
    else:
        prefix, curr, stage, current_gr, cond, fold, seed = parts
    stage=int(stage)
    train_gr = get_traingroups(curr, stage)
    
    tag_dict = {
        'Curriculum':curr,
        'Stage': stage,
        'Condition':cond,
        'Seed': seed,
        'Train Groups': train_gr,
        'data_id': '_'.join([curr, seed, cond])
    }
    return tag_dict

In [11]:
def proc_result_folder_tb(emb_root, ds_task, iter_per_stage, eval_type, 
                       n_jobs=80, exemplar=False):
    if ds_task=='tb_cat':
        evaluator = ToyBoxEval(exemplar=exemplar)
        ds_task = 'category'
    elif ds_task=='tb_trans':
        evaluator = ToyBoxEval(exemplar=exemplar)
        ds_task = 'transformation'
    else:
        raise ValueError('only for toybox')
    record_list = []
    

    fpathlist = [emb_root+fname
                for fname in os.listdir(emb_root)
                if Path(emb_root+fname).suffix=='.csv']
        
    if n_jobs>1:
        record_list = Parallel(n_jobs=n_jobs_external)(
            delayed(job_proc_file)(fp, ds_task, evaluator, iter_per_stage)
            for fp in tqdm(fpathlist)
        )
    else:
        for fp in tqdm(fpathlist):
#             print(fp)
            record = parse_fname(fp)
    #         if '141' in fp:
    #             continue
            record[ds_task] = evaluator.proc_fp(fp, ds_task)
            record['Iteration']=iter_per_stage*record['Stage']
            record_list.append(deepcopy(record))
            
#             break #@@@
    df = pd.DataFrame.from_records(record_list)         
    return df

In [12]:
def proc_result_folder(emb_root, ds_task, iter_per_stage, eval_type, 
                       n_jobs=80):
    if ds_task=='ssv2':
        evaluator = SSv2Eval()
        ds_task = 'category'
    elif ds_task=='tb_cat':
        evaluator = ToyBoxEval()
        ds_task = 'category'
    elif ds_task=='tb_trans':
        evaluator = ToyBoxEval()
        ds_task = 'transformation'
    elif ds_task=='cifar10':
        evaluator = Cifar10Eval()
        ds_task = 'category'
    elif ds_task=='ucf101':
        evaluator = UCF101Eval()
        ds_task = 'category'
        n_jobs_external=n_jobs
        n_jobs_internal=1
    else:
        raise ValueError
    record_list = []
    
    if (ds_task=='tb_cat') |(ds_task=='tb_trans'):
        fpathlist = [emb_root+fname
                    for fname in os.listdir(emb_root)
                    if Path(emb_root+fname).suffix=='.csv']
        
    
    train_test_fp_list = []
    for fname in os.listdir(emb_root):
        if Path(emb_root+fname).suffix!='.csv':
            continue
        train_fp = emb_root+fname
        test_fp = str(Path(emb_root, 'test/', fname))
        
        if not os.path.exists(test_fp):
            print(test_fp, 'does not exist')
            continue
        train_test_fp_list.append(
            {'train':train_fp,
             'test':test_fp})
    

    for fp_dict in tqdm(train_test_fp_list):
#             print(fp)
        record = parse_fname(fp_dict['train'])
        if eval_type=='linear':
            record[ds_task] = evaluator.proc_train_test(fp_dict, ds_task,
                                                       eval_type=eval_type, 
                                                        n_jobs=n_jobs)
        else:
            topkcorrect = evaluator.proc_train_test(fp_dict, ds_task,
                                                       eval_type=eval_type, 
                                                        n_jobs=n_jobs)
            record['Top1'] = topkcorrect[1]
            record['Top5'] = topkcorrect[5]
            record['Top10'] = topkcorrect[10]
        record['Iteration']=iter_per_stage*record['Stage']
        record_list.append(deepcopy(record))
        
    df = pd.DataFrame.from_records(record_list)         
    return df

In [13]:
learner = 'generative/'#'contrastive/'#'predictive/'#
ds_task = 'ssv2'
date = 'sep28'#'aug11'#'aug1'#'jul315'#
num_ep,iter_per_ep = 5, 2000

eval_type='nn'#'linear'#

emb_root = '/N/project/baby_vision_curriculum/trained_models/'+learner+date+'/benchmarks/'+ds_task+'/'
# emb_root = '/N/project/baby_vision_curriculum/trained_models/generative/v3/jul28dev/benchmarks/ssv2/'

iter_per_stage = num_ep*iter_per_ep

df_ss = proc_result_folder(emb_root, ds_task, iter_per_stage, eval_type, n_jobs=23)
# df_ss.to_csv(date+'_'+ds_task+'_'+eval_type+'_score.csv', index=False)
# df_ss.groupby(['Stage', 'Condition', 'Curriculum']).mean()

  3%|█▏                                          | 1/37 [00:01<01:09,  1.93s/it]

Top-1, correct = 0.18, total = 1000, acc = 0.000
Top-5, correct = 0.53, total = 1000, acc = 0.001
Top-10, correct = 0.72, total = 1000, acc = 0.001
Top-20, correct = 0.89, total = 1000, acc = 0.001
Top-50, correct = 0.98, total = 1000, acc = 0.001


  5%|██▍                                         | 2/37 [00:03<01:06,  1.90s/it]

Top-1, correct = 0.10, total = 1000, acc = 0.000
Top-5, correct = 0.42, total = 1000, acc = 0.000
Top-10, correct = 0.64, total = 1000, acc = 0.001
Top-20, correct = 0.85, total = 1000, acc = 0.001
Top-50, correct = 0.99, total = 1000, acc = 0.001


  8%|███▌                                        | 3/37 [00:05<01:04,  1.91s/it]

...

Top-1, correct = 0.14, total = 1000, acc = 0.000
Top-5, correct = 0.46, total = 1000, acc = 0.000
Top-10, correct = 0.67, total = 1000, acc = 0.001
Top-20, correct = 0.88, total = 1000, acc = 0.001
Top-50, correct = 0.99, total = 1000, acc = 0.001


 97%|█████████████████████████████████████████▊ | 36/37 [01:08<00:01,  1.87s/it]

Top-1, correct = 0.24, total = 1000, acc = 0.000
Top-5, correct = 0.58, total = 1000, acc = 0.001
Top-10, correct = 0.76, total = 1000, acc = 0.001
Top-20, correct = 0.89, total = 1000, acc = 0.001
Top-50, correct = 0.97, total = 1000, acc = 0.001


100%|███████████████████████████████████████████| 37/37 [01:09<00:00,  1.89s/it]

Top-1, correct = 0.14, total = 1000, acc = 0.000
Top-5, correct = 0.49, total = 1000, acc = 0.000
Top-10, correct = 0.69, total = 1000, acc = 0.001
Top-20, correct = 0.87, total = 1000, acc = 0.001
Top-50, correct = 0.98, total = 1000, acc = 0.001
